In [21]:
import numpy as np
import pprint as pp

In [22]:
def readRewards():
    R = []
    for line in open("hw9_rewards.txt").readlines():
        R.append(int(line.rstrip()))
    return np.array(R)
    
def readTransitionMat(fname):
    P_a = np.zeros((81,81))
    for line in open(fname).readlines():
        row, col, prob = line.rstrip().split()
        row = [(int(row) - 1)]
        col = [int(col) - 1]
        prob = float(prob)
        P_a[row,col] = prob
        
    return P_a
    
R = readRewards()
P_a1 = readTransitionMat("hw9_prob_a1.txt")
P_a2 = readTransitionMat("hw9_prob_a2.txt")
P_a3 = readTransitionMat("hw9_prob_a3.txt")
P_a4 = readTransitionMat("hw9_prob_a4.txt")
transition_probs = [P_a1, P_a2, P_a3, P_a4]

In [23]:
policy = np.random.randint(4, size=81) # initial policy
gamma = 0.9925
def policy_eval(policy, gamma, transition_probs, R):
    P_policy = []
    for s1 in range(81):
        temp_arr = []
        action = policy[s1]
        for s2 in range(81):
            temp_arr.append(transition_probs[action][s1, s2])
        P_policy.append(temp_arr)
    P_policy = np.array(P_policy)
    return np.dot(np.linalg.inv(np.identity(81) - (gamma * P_policy)),R)

def policy_improv(policy, gamma, transition_probs, R):
    V = policy_eval(policy, gamma, transition_probs, R)
    new_policy = []
    for s1 in range(81):
        maxval = 0
        bestaction = 0
        for action in range(4):
            s = 0
            for s2 in range(81):
                s += transition_probs[action][s1, s2]*V[s2]
            if s > maxval:
                maxval = s
                bestaction = action
        new_policy.append(bestaction)
        
    return (V,np.array(new_policy))

V, new_policy = policy_improv(policy, gamma, transition_probs, R)
while not np.array_equal(new_policy, policy):
    policy = new_policy
    V, new_policy = policy_improv(policy, gamma, transition_probs, R)

In [24]:
new_policy = new_policy.reshape(9,9).T
print(new_policy)

[[0 0 0 0 0 0 0 0 0]
 [0 2 2 3 0 0 3 0 0]
 [2 1 0 3 0 0 3 0 0]
 [0 0 3 0 0 0 3 0 0]
 [0 0 3 0 0 0 3 0 0]
 [0 3 0 0 0 0 3 0 0]
 [0 3 0 2 2 2 2 2 0]
 [0 2 2 1 0 2 2 1 0]
 [0 0 0 0 0 0 0 0 0]]


In [25]:
new_policy = new_policy.tolist()
for i,v in enumerate(V):
    print("State: {}, Value: {}".format(i+1,v))

State: 1, Value: 0.0
State: 2, Value: 0.0
State: 3, Value: 100.70098072748917
State: 4, Value: 0.0
State: 5, Value: 0.0
State: 6, Value: 0.0
State: 7, Value: 0.0
State: 8, Value: 0.0
State: 9, Value: 0.0
State: 10, Value: 0.0
State: 11, Value: 102.37526440102097
State: 12, Value: 101.52364514898133
State: 13, Value: 0.0
State: 14, Value: 0.0
State: 15, Value: 109.48993453646317
State: 16, Value: 110.40903296181372
State: 17, Value: 111.3358466339685
State: 18, Value: 0.0
State: 19, Value: 0.0
State: 20, Value: 103.23462341601055
State: 21, Value: 0.0
State: 22, Value: 106.77826755022942
State: 23, Value: 107.67462642880363
State: 24, Value: 108.57848711681848
State: 25, Value: 0.0
State: 26, Value: 112.27044031794435
State: 27, Value: 0.0
State: 28, Value: 0.0
State: 29, Value: 104.10121204279739
State: 30, Value: 104.97507555494727
State: 31, Value: 105.88853590955107
State: 32, Value: 0.0
State: 33, Value: 0.0
State: 34, Value: 114.16322950263668
State: 35, Value: 113.21287932200805


In [26]:
int_to_direction = {0:'<', 1:'^', 2:'>', 3:'v'}

directions = [[int_to_direction[val] for val in row] for row in new_policy]
pp.pprint(["".join(row) for row in directions])

['<<<<<<<<<',
 '<>>v<<v<<',
 '>^<v<<v<<',
 '<<v<<<v<<',
 '<<v<<<v<<',
 '<v<<<<v<<',
 '<v<>>>>><',
 '<>>^<>>^<',
 '<<<<<<<<<']


In [27]:
V = np.zeros(81)

def val_iteration(V, transition_probs, gamma, R):
    new_V = np.zeros(81)
    policy = np.zeros(81)
    for s1 in range(81):
        maxval = -1*float("inf")
        for action in range(4):
            s = 0
            for s2 in range(81):
                #print("{} {}, {}".format(action,s1, s2))
                s += transition_probs[action][s1, s2]*V[s2]
            if s > maxval:
                maxval = s
                policy[s1] = action
                
        new_V[s1] = (R[s1] + gamma*maxval)
    
    #print("Size of new_V: {}".format(new_V.shape))
    return (new_V, policy)
        
new_V, new_policy = val_iteration(V, transition_probs, gamma, R)
while not np.array_equal(V, new_V):
    V = new_V
    new_V, new_policy = val_iteration(V, transition_probs, gamma, R)


In [28]:
for i,v in np.ndenumerate(new_V):
    print("State: {}, Value: {}".format(i[0]+1,v))

State: 1, Value: 0.0
State: 2, Value: 0.0
State: 3, Value: 100.70098072748767
State: 4, Value: 0.0
State: 5, Value: 0.0
State: 6, Value: 0.0
State: 7, Value: 0.0
State: 8, Value: 0.0
State: 9, Value: 0.0
State: 10, Value: 0.0
State: 11, Value: 102.37526440101946
State: 12, Value: 101.52364514897984
State: 13, Value: 0.0
State: 14, Value: 0.0
State: 15, Value: 109.48993453646155
State: 16, Value: 110.40903296181209
State: 17, Value: 111.33584663396687
State: 18, Value: 0.0
State: 19, Value: 0.0
State: 20, Value: 103.23462341600904
State: 21, Value: 0.0
State: 22, Value: 106.77826755022788
State: 23, Value: 107.67462642880209
State: 24, Value: 108.57848711681693
State: 25, Value: 0.0
State: 26, Value: 112.27044031794271
State: 27, Value: 0.0
State: 28, Value: 0.0
State: 29, Value: 104.10121204279588
State: 30, Value: 104.97507555494578
State: 31, Value: 105.88853590954957
State: 32, Value: 0.0
State: 33, Value: 0.0
State: 34, Value: 114.16322950263502
State: 35, Value: 113.21287932200639

### As we can see from above, the values from policy iteration match the values from value iteration

In [33]:
print(np.array(new_policy).reshape(9,9).T)

[[ 0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  2.  2.  3.  0.  0.  3.  0.  0.]
 [ 2.  1.  0.  3.  0.  0.  3.  0.  0.]
 [ 0.  0.  3.  0.  0.  0.  3.  0.  0.]
 [ 0.  0.  3.  0.  0.  0.  3.  0.  0.]
 [ 0.  3.  0.  0.  0.  0.  3.  0.  0.]
 [ 0.  3.  0.  2.  2.  2.  2.  2.  0.]
 [ 0.  2.  2.  1.  0.  2.  2.  1.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.]]


In [34]:
int_to_direction = {0.0:'<', 1.0:'^', 2.0:'>', 3.0:'v'}

new_policy = np.array(new_policy).reshape(9,9).T.tolist()
directions = [[int_to_direction[val] for val in row] for row in new_policy]
pp.pprint(["".join(row) for row in directions])

['<<<<<<<<<',
 '<>>v<<v<<',
 '>^<v<<v<<',
 '<<v<<<v<<',
 '<<v<<<v<<',
 '<v<<<<v<<',
 '<v<>>>>><',
 '<>>^<>>^<',
 '<<<<<<<<<']
